In [89]:
import pandas as pd
import os
import re
from datetime import datetime
import pdfplumber

In [90]:
Payout_Summary_Order_Level_Sales_folder_path = r"C:\Users\jayas\Desktop\cleaningAndmerging\Payout Summary & Order Level Sales"
Commission_invoices_files=os.listdir(r"C:\Users\jayas\Desktop\cleaningAndmerging\Commission Invoices")[1:]
invoice_Annexure_files = os.listdir(Payout_Summary_Order_Level_Sales_folder_path)

In [91]:
summary_template = pd.read_excel(r"C:\Users\jayas\Desktop\cleaningAndmerging\Data Analyst Quick Assignment.xlsx", sheet_name="Summary")
payout_breakout_template = pd.read_excel(r"C:\Users\jayas\Desktop\cleaningAndmerging\Data Analyst Quick Assignment.xlsx", sheet_name="Payout Breakup Tab")
order_level_template = pd.read_excel(r"C:\Users\jayas\Desktop\cleaningAndmerging\Data Analyst Quick Assignment.xlsx", sheet_name="Order Level")
commission_invoice_template = pd.read_excel(r"C:\Users\jayas\Desktop\cleaningAndmerging\Commission Invoices\Swiggy_Tax_Sample_file.xlsx")

required_columns_order_level = order_level_template.columns.to_list()
required_columns_summary = summary_template.columns.to_list()
required_columns_payout_breakout = payout_breakout_template.columns.to_list()
required_columns_commission_invoice = commission_invoice_template.columns.to_list()

In [92]:
import fitz

def extract_text_with_pymupdf(file_path):
    document = fitz.open(file_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text


# Helper to format date range
def format_date_range(start, end):
    start_dt = datetime.strptime(start, "%d/%m/%Y").strftime("%d %B %Y")
    end_dt = datetime.strptime(end, "%d/%m/%Y").strftime("%d %B %Y")
    return f"{start_dt} - {end_dt}"

In [93]:
def get_commission_df(file_path,file_name,count):

    # Original OCR text of the invoic
    invoice_text = extract_text_with_pymupdf(file_path)

    # Extract fields using regex
    part1_dict ={}


    invoice_number = re.search(r"Invoice Number\s*:\s*(\w+)", invoice_text).group(1)
    invoice_date_raw = re.search(r"Invoice Date\s*:\s*([\d\-: ]+)", invoice_text).group(1)
    invoice_date = datetime.strptime(invoice_date_raw, "%Y-%m-%d %H:%M:%S")
    fy_year = f"FY_{invoice_date.year - 1}-{invoice_date.year}" if invoice_date.month < 4 else f"FY_{invoice_date.year}-{invoice_date.year + 1}"
    month_str = f"{invoice_date.month:02d}_{invoice_date.strftime('%B')}"
    payout_start, payout_end = re.search(r"Service Period\s*:\s*(\d{2}/\d{2}/\d{4}) to (\d{2}/\d{2}/\d{4})", invoice_text).groups()
    payout_period = format_date_range(payout_start, payout_end)
    irn = re.search(r"IRN\s*:\s*([a-f0-9]+)", invoice_text).group(1)
    mann_gstin = re.search(r"Restaurant / Store ID\s*:\s*\d+\nGSTIN\s*:\s*([\w\d]+)", invoice_text).group(1)
    swiggy_gstin = re.search(r"GSTIN\s*:\s*(29AAFCB7707D1ZQ)", invoice_text).group(1)

    other_charges = re.search(r"Other Charges\s*-\s*Reimbursement\s*of\s*Discount\s*([\d.,]+)", invoice_text).group(1)
    grand_total = re.search(r"Grand Total\s*([\d.,]+)", invoice_text).group(1)
    store_id = re.search(r"Restaurant / Store ID\s*:\s*(\d+)", invoice_text).group(1)
    pan = re.search(r"PAN\s*:\s*([A-Z]{5}\d{4}[A-Z])", invoice_text).group(1)
    invoice_type = re.search(r"Invoice Type\s*:\s*(\w+)", invoice_text).group(1)

    part1_dict['id'] = count
    part1_dict['payout_period'] = payout_period
    part1_dict['file_name'] = file_name
    part1_dict['fy_year'] = fy_year
    part1_dict['year'] = str(invoice_date.year)
    part1_dict['month'] = month_str
    part1_dict['irn'] = irn
    part1_dict['mann_gstin'] = mann_gstin
    part1_dict['swiggy_gstin'] = swiggy_gstin



    part1_df = pd.DataFrame([part1_dict], columns = required_columns_commission_invoice[:required_columns_commission_invoice.index('sr_no')])



#Extracting table
    tables = []
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            table = page.extract_table()
            if table:
                tables.append(table)
            
    table_df = pd.DataFrame(tables[0]).dropna(how='any')
    table_cols = required_columns_commission_invoice[required_columns_commission_invoice.index('sr_no'):required_columns_commission_invoice.index('total_amount')+1]
    table_df.columns=table_cols
    table_df = table_df.reset_index(drop='index').loc[1:][:2]


    part2_dict = {}


    part2_dict['other_charges_reimbursement_of_discount']=other_charges
    part2_dict['grand_total']=grand_total
    part2_dict['brand_id']=store_id
    part2_dict['pan']=pan
    part2_dict['invoice_date']=invoice_date
    part2_dict['invoice_number']=invoice_number
    part2_dict['original_invoice_number']="nan"
    part2_dict['invoice_type']=invoice_type
    part2_dict['original_invoice_date']="nan"



    part2_df = pd.DataFrame([part2_dict], columns = required_columns_commission_invoice[required_columns_commission_invoice.index('total_amount')+1:])

    df1_repeated = pd.concat([part1_df]*len(table_df)).reset_index(drop=True)
    df2_repeated = pd.concat([part2_df]*len(table_df)).reset_index(drop=True)
    df3 = table_df.reset_index(drop=True)


    commission_invoice_final_df = pd.concat([df1_repeated, df2_repeated, df3], axis=1)


    return commission_invoice_final_df


In [ ]:
count = 0

summary_data_list=[]
payout_breakup_data_list=[]
order_level_df= pd.DataFrame(columns=required_columns_order_level)
commisiong_invoice_df = pd.DataFrame(columns=required_columns_commission_invoice)

for file in invoice_Annexure_files:
    count+=1
    ##Read Files
    file_path = os.path.join(Payout_Summary_Order_Level_Sales_folder_path,file)
    raw_data_summary = pd.read_excel(file_path,header=None,sheet_name="Summary")
    raw_payout_breakup = pd.read_excel(file_path,header=None,sheet_name="Payout Breakup")
    raw_data_order_level = pd.read_excel(file_path,header=None,sheet_name="Order Level")

    ##Initial transformation
    raw_data_summary_list = raw_data_summary.dropna(how='all',axis=1).dropna(how="all").astype(str).apply(lambda row: "|".join(cell.strip() for cell in row if cell.strip()), axis=1).to_list()
    raw_payout_breakup_list = raw_payout_breakup.dropna(how='all',axis=1).dropna(how="all").astype(str).apply(lambda row: "|".join(cell.strip() for cell in row if cell.strip()), axis=1).to_list()
    
    ## Summary Extraction
    first_5 = raw_data_summary_list[:4]
    remaining_data = [i for i in raw_data_summary_list if i.split('|')[0] in required_columns_summary ]


    summary_final_data = {}
    payout_breakup_final_data = {}

    for key,value in zip(required_columns_summary[:4],first_5):
        if "Rest. ID" in value:
            value=value.split('-')[1]
        summary_final_data[key]= value.split("|")[0].strip(" ")

    for i in remaining_data:
        key = i.split('|')[0]
        value = i.split('|')[1]
        summary_final_data[key] = value

    summary_final_data["File Name"] = file
    summary_data_list.append(summary_final_data)
    summary_data_final_df = pd.DataFrame(summary_data_list,columns=required_columns_summary)

    ##Payout Breakup Extraction

    order_values = [i.replace('nan',"").split('|')[2:] for i in raw_payout_breakup_list if 'Orders' in i.split('|')][0]

    payout_breakup_final_data['SR.No'] = count
    payout_breakup_final_data['Particulars'] = 'Orders'

    for k,v in zip(['Delivered Orders','Cancelled Orders','Total'],order_values):
        payout_breakup_final_data[k]=v

    for k in ['Brand','Res-Id','Payout Period','File Name']:
        payout_breakup_final_data[k]=summary_final_data[k]

    payout_breakup_data_list.append(payout_breakup_final_data)

    payout_breakout_final_df = pd.DataFrame(payout_breakup_data_list,columns=required_columns_payout_breakout)

    ##Order Level extraction

    for idx,k in enumerate(['Brand','Res-Id','Payout Period','File Name']):
        raw_data_order_level.insert(idx,k,summary_final_data[k])

    raw_data_order_level.columns = required_columns_order_level
    order_level_local_df = raw_data_order_level[3:].reset_index(drop='index')
    order_level_df = pd.concat([order_level_df, order_level_local_df])


    ##Commission invoice extraction

count = 0
    
for file in Commission_invoices_files[1:]:
    count+=1
    file_path = os.path.join(r"C:\Users\jayas\Desktop\cleaningAndmerging\Commission Invoices",file)
    commisiong_invoice_local_df = get_commission_df(file_path,file,count)
    commisiong_invoice_df = pd.concat([commisiong_invoice_df, commisiong_invoice_local_df])


summary_data_final_df.to_excel("Summary_data.xlsx",index=False,sheet_name="Summary_data")
payout_breakout_final_df.to_excel("Payout Breakup.xlsx",index=False,sheet_name="Payout Breakup")
order_level_df.to_excel("Order Level.xlsx",index=False,sheet_name="Order Level")
commisiong_invoice_df.to_excel("Commission invoice.xlsx",index=False,sheet_name="Commission invoice")
